In [1]:
from selenium import webdriver
import time,re,random,sys
import pandas as pd

driver = webdriver.Chrome()
driver.set_window_size(400,1000)

In [2]:
attitude_keyurl = r'<a href=".{4,5}://weibo.cn/attitude/.*?</a>'
repost_keyurl = r'<a href=".{4,5}://weibo.cn/repost/.*?</a>'
comment_keyurl = r'<a href=".{4,5}://weibo.cn/comment/.{45,52}>评论.*?</a>'
def makeattitudeurllist(x):
    uum = weibodf.iloc[x]
    attitude_url = ['https://weibo.cn/attitude/'+uum['id']+'?rl=1&page='+str(x) 
                    for x in range(1,int(uum['attitude_num'])//10+2,1)]
    return(attitude_url)
def makereposturllist(x):
    uum = weibodf.iloc[x]
    repost_url = ['https://weibo.cn/repost/'+uum['id']+'?uid='+uum['uid']+'&&page='+str(x) 
                  for x in range(1,int(uum['repost_num'])//10+2,1)]
    return(repost_url)
def makecommenturllist(x):
    uum = weibodf.iloc[x]
    comment_url = ['https://weibo.cn/comment/'+uum['id']+'?&uid='+uum['uid']+'&&page='+str(x) 
                   for x in range(1,int(uum['comment_num'])//10+2,1)]
    return(comment_url)

def makedf():
    global weibodf
    weibodf.index = range(len(weibodf))
    weibodf['No'] = range(len(weibodf))
    weibodf['id'] = weibodf['repost'].map(lambda x:re.findall('[A-Za-z][A-Z0-9a-z]{8}',x)[0])
    weibodf['uid'] = weibodf['repost'].map(lambda x:re.findall('[0-9]{8,10}',x)[0])
    weibodf['attitude_num'] = weibodf['attitude'].map(lambda x:re.findall(r'赞.*?<',x)[0][2:-2])
    weibodf['repost_num'] = weibodf['repost'].map(lambda x:re.findall(r'转发.*?<',x)[0][3:-2])
    weibodf['comment_num'] = weibodf['comment'].map(lambda x:re.findall(r'评论.*?<',x)[0][3:-2])
    weibodf['attitude_list'] = weibodf['No'].map(makeattitudeurllist)
    weibodf['repost_list'] = weibodf['No'].map(makereposturllist)
    weibodf['comment_list'] = weibodf['No'].map(makecommenturllist)
    weibodf['all_list'] = weibodf['attitude_list']+weibodf['comment_list']#+weibodf['repost_list']

In [3]:
ordernm = 0
zhdf = pd.read_excel('zhmm.xlsx')
zhanghao,mima = zhdf.iloc[ordernm]['zhanghao'],zhdf.iloc[ordernm]['mima']
driver.get('https://passport.weibo.cn/signin/login?');time.sleep(3)
elem = driver.find_element_by_id('loginName')
elem.send_keys(zhanghao)
elem = driver.find_element_by_id('loginPassword')
elem.send_keys(mima)
driver.find_element_by_id('loginAction').click();time.sleep(5)

In [ ]:
for i in [
'6171388940',
'5615627418',
'5350509769',
'2447034645'
]:
    topicname = 'ID'+i
    homepage = r'https://weibo.cn/u/%s?page=' % i
    
    weibodf = pd.DataFrame()
    uidlist = pd.Series()
    for homelist in range(1,3,1):
        url = homepage+str(homelist)
        driver.get(url)
        time.sleep(1.5)
        df = pd.DataFrame({
            'attitude':re.findall(attitude_keyurl,driver.page_source),
            'repost':re.findall(repost_keyurl,driver.page_source),
            'comment':re.findall(comment_keyurl,driver.page_source)})
        if homelist == 1:
            df = df.iloc[1:]
        weibodf = pd.concat([weibodf,df]).drop_duplicates()

    makedf()
    llist = []
    for x in weibodf['all_list']:
        for y in x:
            llist.append(y)
    mar = 0
    del(weibodf)
    print(topicname)
    print(len(llist))

    while mar < len(llist):
        driver.get(llist[mar])
        time.sleep(random.randint(200,300)/100)
        uid = pd.Series(re.findall(r'/u/[0-9]{8,10}',driver.page_source))
        uid = uid.map(lambda x:x[3:])
        uidlist = pd.concat([uidlist,uid])
        if len(uid) < 2:mar += random.randint(2,6)
        else: mar += 1        
        sys.stdout.write('表序数 %s--ID总数量 %s\r'%(mar,len(uidlist)))
        sys.stdout.flush()
        if mar > random.randint(30000,30123):
            break
    
    uidlist.drop_duplicates() \
    .to_csv('%s-%s-%s-%s.csv'%(mar*10,len(uidlist),
                                          len(uidlist.drop_duplicates()),topicname),index=False,encoding='utf-8')

ID6171388940
30563


In [ ]:
driver.close()